# 101 Pandas Exercises for Data Analysis

## Index
#### 31. How to fill an intermittent time series so all missing dates show up with values of previous non-missing date?
#### 32. How to compute the autocorrelations of a numeric series?
#### 33. How to import only every nth row from a csv file to create a dataframe?
#### 34. How to change column values when importing csv to a dataframe?
#### 35. How to create a dataframe with rows as strides from a given series?
#### 36. How to import only specified columns from a csv file?
#### 37. How to get the nrows, ncolumns, datatype, summary stats of each column of a dataframe? Also get the array and list equivalent.
#### 38. How to extract the row and column number of a particular cell with given criterion?
#### 39. How to rename a specific columns in a dataframe?
#### 40. How to check if a dataframe has any missing values?


## 31. How to fill an intermittent time series so all missing dates show up with values of previous non-missing date?

In [1]:
import pandas as pd
import numpy as np
ser = pd.Series([1,10,3,np.nan], index=pd.to_datetime(['2000-01-01', '2000-01-03', '2000-01-06', '2000-01-08']))
print(ser)

2000-01-01     1.0
2000-01-03    10.0
2000-01-06     3.0
2000-01-08     NaN
dtype: float64


In [2]:
# Solution
ser.resample('D').ffill()  # fill with previous value


2000-01-01     1.0
2000-01-02     1.0
2000-01-03    10.0
2000-01-04    10.0
2000-01-05    10.0
2000-01-06     3.0
2000-01-07     3.0
2000-01-08     NaN
Freq: D, dtype: float64

In [3]:
# Alternatives
ser.resample('D').bfill()  # fill with next value


2000-01-01     1.0
2000-01-02    10.0
2000-01-03    10.0
2000-01-04     3.0
2000-01-05     3.0
2000-01-06     3.0
2000-01-07     NaN
2000-01-08     NaN
Freq: D, dtype: float64

In [4]:
ser.resample('D').bfill().ffill()  # fill next else prev value

2000-01-01     1.0
2000-01-02    10.0
2000-01-03    10.0
2000-01-04     3.0
2000-01-05     3.0
2000-01-06     3.0
2000-01-07     3.0
2000-01-08     3.0
Freq: D, dtype: float64

## 32. How to compute the autocorrelations of a numeric series?

In [5]:
# Compute autocorrelations for the first 10 lags of ser. Find out which lag has the largest correlation.
ser = pd.Series(np.arange(20) + np.random.normal(1, 10, 20))
ser

0      2.576939
1     -5.925263
2      8.662783
3     -7.528752
4    -17.891275
5      6.773157
6     18.592421
7     15.650434
8      3.072392
9     15.558102
10    -3.400781
11    -0.506777
12    -1.197325
13    -1.042822
14   -19.727668
15     7.833700
16    15.001276
17    24.414241
18    -2.648138
19    23.528737
dtype: float64

In [6]:
# Solution
autocorrelations = [ser.autocorr(i).round(2) for i in range(11)]
print(autocorrelations[1:])
print('Lag having highest correlation: ', np.argmax(np.abs(autocorrelations[1:]))+1)

[0.12, 0.03, -0.15, 0.03, -0.65, -0.22, -0.13, -0.06, 0.06, 0.87]
Lag having highest correlation:  10


## 33. How to import only every nth row from a csv file to create a dataframe?

In [55]:
# Solution 1: Use chunks and for-loop
df = pd.read_csv('BostonHousing.csv', chunksize=50)
df2 = pd.DataFrame()
for chunk in df:
    df2 = df2.append(chunk.iloc[0,:])

df2



,age,b,chas,crim,dis,indus,lstat,medv,nox,ptratio,rad,rm,tax,zn
0,65.2,396.90,0.0,0.00632,4.0900,2.31,4.98,24.0,0.538,15.3,1.0,6.575,296.0,18.0
50,45.7,395.56,0.0,0.08873,6.8147,5.64,13.45,19.7,0.439,16.8,4.0,5.963,243.0,21.0
100,79.9,394.76,0.0,0.14866,2.7778,8.56,9.42,27.5,0.520,20.9,5.0,6.727,384.0,0.0
150,97.3,372.80,0.0,1.65660,1.6180,19.58,14.10,21.5,0.871,14.7,5.0,6.122,403.0,0.0
200,13.9,384.30,0.0,0.01778,7.6534,1.47,4.45,32.9,0.403,17.0,3.0,7.135,402.0,95.0
250,13.0,396.28,0.0,0.14030,7.3967,5.86,5.90,24.4,0.431,19.1,7.0,6.487,330.0,22.0
300,47.4,390.86,0.0,0.04417,7.8278,2.24,6.07,24.8,0.400,14.8,5.0,6.871,358.0,70.0
350,44.4,396.90,0.0,0.06211,8.7921,1.25,5.98,22.9,0.429,19.7,1.0,6.490,335.0,40.0
400,100.0,396.90,0.0,25.04610,1.5888,18.10,26.77,5.6,0.693,20.2,24.0,5.987,666.0,0.0
450,92.6,0.32,0.0,6.71772,2.3236,18.10,17.44,13.4,0.713,20.2,24.0,6.749,666.0,0.0


In [54]:
# Solution 2: Use chunks and list comprehension
df = pd.read_csv('BostonHousing.csv', chunksize=50)
df2 = pd.concat([chunk.iloc[0] for chunk in df], axis=1)
df2 = df2.transpose()
df2

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
50,0.08873,21.0,5.64,0.0,0.439,5.963,45.7,6.8147,4.0,243.0,16.8,395.56,13.45,19.7
100,0.14866,0.0,8.56,0.0,0.520,6.727,79.9,2.7778,5.0,384.0,20.9,394.76,9.42,27.5
150,1.65660,0.0,19.58,0.0,0.871,6.122,97.3,1.6180,5.0,403.0,14.7,372.80,14.10,21.5
200,0.01778,95.0,1.47,0.0,0.403,7.135,13.9,7.6534,3.0,402.0,17.0,384.30,4.45,32.9
250,0.14030,22.0,5.86,0.0,0.431,6.487,13.0,7.3967,7.0,330.0,19.1,396.28,5.90,24.4
300,0.04417,70.0,2.24,0.0,0.400,6.871,47.4,7.8278,5.0,358.0,14.8,390.86,6.07,24.8
350,0.06211,40.0,1.25,0.0,0.429,6.490,44.4,8.7921,1.0,335.0,19.7,396.90,5.98,22.9
400,25.04610,0.0,18.10,0.0,0.693,5.987,100.0,1.5888,24.0,666.0,20.2,396.90,26.77,5.6
450,6.71772,0.0,18.10,0.0,0.713,6.749,92.6,2.3236,24.0,666.0,20.2,0.32,17.44,13.4


In [33]:
# Solution 3: Use csv reader
import csv          
with open('BostonHousing.csv', 'r') as f:
    reader = csv.reader(f)
    out = []
    for i, row in enumerate(reader):
        if i%50 == 0:
            out.append(row)

df2 = pd.DataFrame(out[1:], columns=out[0])
print(df2.head())

      crim  zn  indus chas    nox     rm   age     dis rad  tax ptratio  \
0  0.21977   0   6.91    0  0.448  5.602    62  6.0877   3  233    17.9   
1   0.0686   0   2.89    0  0.445  7.416  62.5  3.4952   2  276      18   
2  2.73397   0  19.58    0  0.871  5.597  94.9  1.5257   5  403    14.7   
3   0.0315  95   1.47    0  0.403  6.975  15.3  7.6534   3  402      17   
4  0.19073  22   5.86    0  0.431  6.718  17.5  7.8265   7  330    19.1   

        b  lstat  medv  
0   396.9   16.2  19.4  
1   396.9   6.19  33.2  
2  351.85  21.45  15.4  
3   396.9   4.56  34.9  
4  393.74   6.56  26.2  


## 34. How to change column values when importing csv to a dataframe?

In [34]:
# Solution 2: Using csv reader
import csv
with open('BostonHousing.csv', 'r') as f:
    reader = csv.reader(f)
    out = []
    for i, row in enumerate(reader):
        if i > 0:
            row[13] = 'High' if float(row[13]) > 25 else 'Low'
        out.append(row)

df = pd.DataFrame(out[1:], columns=out[0])
print(df.head())
    
    

      crim  zn indus chas    nox     rm   age     dis rad  tax ptratio  \
0  0.00632  18  2.31    0  0.538  6.575  65.2    4.09   1  296    15.3   
1  0.02731   0  7.07    0  0.469  6.421  78.9  4.9671   2  242    17.8   
2  0.02729   0  7.07    0  0.469  7.185  61.1  4.9671   2  242    17.8   
3  0.03237   0  2.18    0  0.458  6.998  45.8  6.0622   3  222    18.7   
4  0.06905   0  2.18    0  0.458  7.147  54.2  6.0622   3  222    18.7   

        b lstat  medv  
0   396.9  4.98   Low  
1   396.9  9.14   Low  
2  392.83  4.03  High  
3  394.63  2.94  High  
4   396.9  5.33  High  


In [19]:
# Solution 2
from collections import Counter
mask = ser.map(lambda x: sum([Counter(x.lower()).get(i, 0) for i in list('aeiou')]) >= 2)
ser[mask]

0     Apple
1    Orange
4     Money
dtype: object

## 35. How to create a dataframe with rows as strides from a given series?

In [35]:
L = pd.Series(range(15))
L

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
dtype: int64

In [36]:
def gen_strides(a, stride_len=5, window_len=5):
    n_strides = ((a.size-window_len)//stride_len) + 1
    return np.array([a[s:(s+window_len)] for s in np.arange(0, a.size, stride_len)[:n_strides]])

gen_strides(L, stride_len=2, window_len=4)

array([[ 0,  1,  2,  3],
       [ 2,  3,  4,  5],
       [ 4,  5,  6,  7],
       [ 6,  7,  8,  9],
       [ 8,  9, 10, 11],
       [10, 11, 12, 13]], dtype=int64)

## 36. How to import only specified columns from a csv file?

In [53]:
#Import ‘crim’ and ‘medv’ columns of the BostonHousing dataset as a dataframe.

df = pd.read_csv('BostonHousing.csv', usecols=['crim', 'medv'])
print(df.head())

      crim  medv
0  0.00632  24.0
1  0.02731  21.6
2  0.02729  34.7
3  0.03237  33.4
4  0.06905  36.2


## 37. How to get the nrows, ncolumns, datatype, summary stats of each column of a dataframe? Also get the array and list equivalent.

In [52]:
df = pd.read_csv('Cars93_miss.csv')

#  number of rows and columns
print("Shape",df.shape)
print("********")
# datatypes
print("Dtype",df.dtypes)
print("********")
print("Value_Count",df.dtypes.value_counts())
print("********")
# how many columns under each dtype
print("Dtype_Count",df.get_dtype_counts())

print("********")
# summary statistics
df_stats = df.describe()
df_stats.T


Shape (93, 27)
********
Dtype Manufacturer           object
Model                  object
Type                   object
Min.Price             float64
Price                 float64
Max.Price             float64
MPG.city              float64
MPG.highway           float64
AirBags                object
DriveTrain             object
Cylinders              object
EngineSize            float64
Horsepower            float64
RPM                   float64
Rev.per.mile          float64
Man.trans.avail        object
Fuel.tank.capacity    float64
Passengers            float64
Length                float64
Wheelbase             float64
Width                 float64
Turn.circle           float64
Rear.seat.room        float64
Luggage.room          float64
Weight                float64
Origin                 object
Make                   object
dtype: object
********
Value_Count float64    18
object      9
dtype: int64
********
Dtype_Count float64    18
object      9
dtype: int64
********


,count,mean,std,min,25%,50%,75%,max
Min.Price,86.0,17.118605,8.828290,6.7,10.825,14.60,20.250,45.4
Price,91.0,19.616484,9.724280,7.4,12.350,17.70,23.500,61.9
Max.Price,88.0,21.459091,10.696563,7.9,14.575,19.15,24.825,80.0
MPG.city,84.0,22.404762,5.841520,15.0,18.000,21.00,25.000,46.0
MPG.highway,91.0,29.065934,5.370293,20.0,26.000,28.00,31.000,50.0
EngineSize,91.0,2.658242,1.045845,1.0,1.800,2.30,3.250,5.7
Horsepower,86.0,144.000000,53.455204,55.0,100.750,140.00,170.000,300.0
RPM,90.0,5276.666667,605.554811,3800.0,4800.000,5200.00,5787.500,6500.0
Rev.per.mile,87.0,2355.000000,486.916616,1320.0,2017.500,2360.00,2565.000,3755.0
Fuel.tank.capacity,85.0,16.683529,3.375748,9.2,14.500,16.50,19.000,27.0


In [47]:
# numpy array 
df_arr = df.values
print(df_arr)


array([['Acura', 'Integra', 'Small', ..., 2705.0, 'non-USA',
        'Acura Integra'],
       [nan, 'Legend', 'Midsize', ..., 3560.0, 'non-USA', 'Acura Legend'],
       ['Audi', '90', 'Compact', ..., 3375.0, 'non-USA', 'Audi 90'],
       ...,
       ['Volkswagen', 'Corrado', 'Sporty', ..., 2810.0, 'non-USA',
        'Volkswagen Corrado'],
       ['Volvo', '240', 'Compact', ..., 2985.0, 'non-USA', 'Volvo 240'],
       [nan, '850', 'Midsize', ..., 3245.0, 'non-USA', 'Volvo 850']],
      dtype=object)

In [48]:
# list
df_list = df.values.tolist()
print(df_list)

[['Acura', 'Integra', 'Small', 12.9, 15.9, 18.8, 25.0, 31.0, 'None', 'Front', '4', 1.8, 140.0, 6300.0, 2890.0, 'Yes', 13.2, 5.0, 177.0, 102.0, 68.0, 37.0, 26.5, nan, 2705.0, 'non-USA', 'Acura Integra'], [nan, 'Legend', 'Midsize', 29.2, 33.9, 38.7, 18.0, 25.0, 'Driver & Passenger', 'Front', '6', 3.2, 200.0, 5500.0, 2335.0, 'Yes', 18.0, 5.0, 195.0, 115.0, 71.0, 38.0, 30.0, 15.0, 3560.0, 'non-USA', 'Acura Legend'], ['Audi', '90', 'Compact', 25.9, 29.1, 32.3, 20.0, 26.0, 'Driver only', 'Front', '6', 2.8, 172.0, 5500.0, 2280.0, 'Yes', 16.9, 5.0, 180.0, 102.0, 67.0, 37.0, 28.0, 14.0, 3375.0, 'non-USA', 'Audi 90'], ['Audi', '100', 'Midsize', nan, 37.7, 44.6, 19.0, 26.0, 'Driver & Passenger', nan, '6', nan, 172.0, 5500.0, 2535.0, nan, 21.1, 6.0, 193.0, 106.0, nan, 37.0, 31.0, 17.0, 3405.0, 'non-USA', 'Audi 100'], ['BMW', '535i', 'Midsize', nan, 30.0, nan, 22.0, 30.0, nan, 'Rear', '4', 3.5, 208.0, 5700.0, 2545.0, 'Yes', 21.1, 4.0, 186.0, 109.0, 69.0, 39.0, 27.0, 13.0, 3640.0, 'non-USA', 'BMW 53

## 38. How to extract the row and column number of a particular cell with given criterion?

In [57]:
#Which manufacturer, model and type has the highest Price? 
#What is the row and column number of the cell with the highest "Price" value?

df = pd.read_csv('Cars93_miss.csv')
df.head()

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25.0,31.0,None,Front,...,5.0,177.0,102.0,68.0,37.0,26.5,NaN,2705.0,non-USA,Acura Integra
1,NaN,Legend,Midsize,29.2,33.9,38.7,18.0,25.0,Driver & Passenger,Front,...,5.0,195.0,115.0,71.0,38.0,30.0,15.0,3560.0,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20.0,26.0,Driver only,Front,...,5.0,180.0,102.0,67.0,37.0,28.0,14.0,3375.0,non-USA,Audi 90
3,Audi,100,Midsize,NaN,37.7,44.6,19.0,26.0,Driver & Passenger,NaN,...,6.0,193.0,106.0,NaN,37.0,31.0,17.0,3405.0,non-USA,Audi 100
4,BMW,535i,Midsize,NaN,30.0,NaN,22.0,30.0,NaN,Rear,...,4.0,186.0,109.0,69.0,39.0,27.0,13.0,3640.0,non-USA,BMW 535i


In [69]:
# Solution
# Get Manufacturer with highest price
a = df.loc[df.Price == np.max(df.Price), ['Manufacturer', 'Model', 'Type']]
print(a)



     Manufacturer Model     Type
58  Mercedes-Benz  300E  Midsize


In [63]:
# Get Row and Column number
row, col = np.where(df.values == np.max(df.Price))
print(row)
print(col)

[58]
[4]


In [66]:
# Get the value
b = df.iat[row[0], col[0]]
c = df.iloc[row[0], col[0]]

print(b)
print(c)

61.9
61.9


In [67]:
# Alternates
d = df.at[row[0], 'Price']
e = df.get_value(row[0], 'Price')

print(d)
print(e)

61.9
61.9


D:\Program_Files\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


## 39. How to rename a specific columns in a dataframe?

In [70]:
# Rename the column Type as CarType in df and replace the ‘.’ in column names with ‘_’.

df = pd.read_csv('Cars93_miss.csv')
print(df.columns)

Index(['Manufacturer', 'Model', 'Type', 'Min.Price', 'Price', 'Max.Price',
       'MPG.city', 'MPG.highway', 'AirBags', 'DriveTrain', 'Cylinders',
       'EngineSize', 'Horsepower', 'RPM', 'Rev.per.mile', 'Man.trans.avail',
       'Fuel.tank.capacity', 'Passengers', 'Length', 'Wheelbase', 'Width',
       'Turn.circle', 'Rear.seat.room', 'Luggage.room', 'Weight', 'Origin',
       'Make'],
      dtype='object')


In [71]:
df.columns = df.columns.str.replace(".", "_")
df.columns

Index(['Manufacturer', 'Model', 'Type', 'Min_Price', 'Price', 'Max_Price',
       'MPG_city', 'MPG_highway', 'AirBags', 'DriveTrain', 'Cylinders',
       'EngineSize', 'Horsepower', 'RPM', 'Rev_per_mile', 'Man_trans_avail',
       'Fuel_tank_capacity', 'Passengers', 'Length', 'Wheelbase', 'Width',
       'Turn_circle', 'Rear_seat_room', 'Luggage_room', 'Weight', 'Origin',
       'Make'],
      dtype='object')

In [73]:
# Solution
# Step 1:
df = df.rename(columns = {'Type':'CarType'})
df.columns

Index(['Manufacturer', 'Model', 'CarType', 'Min_Price', 'Price', 'Max_Price',
       'MPG_city', 'MPG_highway', 'AirBags', 'DriveTrain', 'Cylinders',
       'EngineSize', 'Horsepower', 'RPM', 'Rev_per_mile', 'Man_trans_avail',
       'Fuel_tank_capacity', 'Passengers', 'Length', 'Wheelbase', 'Width',
       'Turn_circle', 'Rear_seat_room', 'Luggage_room', 'Weight', 'Origin',
       'Make'],
      dtype='object')

## 40. How to check if a dataframe has any missing values?

In [74]:
df = pd.read_csv('Cars93_miss.csv')
df.head()

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25.0,31.0,None,Front,...,5.0,177.0,102.0,68.0,37.0,26.5,NaN,2705.0,non-USA,Acura Integra
1,NaN,Legend,Midsize,29.2,33.9,38.7,18.0,25.0,Driver & Passenger,Front,...,5.0,195.0,115.0,71.0,38.0,30.0,15.0,3560.0,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20.0,26.0,Driver only,Front,...,5.0,180.0,102.0,67.0,37.0,28.0,14.0,3375.0,non-USA,Audi 90
3,Audi,100,Midsize,NaN,37.7,44.6,19.0,26.0,Driver & Passenger,NaN,...,6.0,193.0,106.0,NaN,37.0,31.0,17.0,3405.0,non-USA,Audi 100
4,BMW,535i,Midsize,NaN,30.0,NaN,22.0,30.0,NaN,Rear,...,4.0,186.0,109.0,69.0,39.0,27.0,13.0,3640.0,non-USA,BMW 535i


In [79]:
df.isnull().values.any()

True